## The Android App Market Analysis
 A comprehensive analysis of the Android app market in order to devise strategies to drive growth and retention.
 
Based on datasets from the the Google Play website. 

apps.csv contains all the details of the applications on Google Play. There are 13 features that describe a given app.

In [23]:
import pandas as pd
import numpy as np
import plotly
plotly.offline.init_notebook_mode(connected=True)
import plotly.graph_objs as go

apps = pd.read_csv("datasets/apps.csv", index_col=0)

apps.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19.0,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14.0,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25.0,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [24]:
# A random sample of 5 rows
print(apps.sample(5))

                                                 App       Category  Rating  \
359                                     Google Voice  COMMUNICATION     4.2   
7796                  Deck Builder & Analyzer for CR         FAMILY     3.1   
8957                            DV Lottery Simulator         FAMILY     4.6   
7084                               CA Latam Partners       BUSINESS     NaN   
2002  Marble Woka Woka 2018 - Bubble Shooter Match 3           GAME     4.6   

      Reviews  Size     Installs  Type Price Content Rating         Genres  \
359    171031   NaN  10,000,000+  Free     0       Everyone  Communication   
7796       52   9.8       5,000+  Free     0       Everyone  Entertainment   
8957       13   1.3       1,000+  Free     0       Everyone  Entertainment   
7084        0  12.0          10+  Free     0       Everyone       Business   
2002   155186   NaN  10,000,000+  Free     0       Everyone         Puzzle   

          Last Updated         Current Ver         Andro

# Clearing data

Let's turn columns "Installs" and "Price" into purely numeric:

In [25]:
# List of characters to remove
chars_to_remove = ['+', ',', '$']
# List of column names to clean
cols_to_clean = ['Installs', 'Price']

for col in cols_to_clean:
    for char in chars_to_remove:
        apps[col] = apps[col].apply(lambda x: x.replace(char, ''))
    apps[col] = apps[col].astype('float')

print(apps.dtypes)

App                object
Category           object
Rating            float64
Reviews             int64
Size              float64
Installs          float64
Type               object
Price             float64
Content Rating     object
Genres             object
Last Updated       object
Current Ver        object
Android Ver        object
dtype: object


# Exploring app categories

In [41]:
num_categories = apps['Category'].nunique()
print('Number of categories = ', num_categories)

# Count the number of apps in each 'Category'. 
num_apps_in_category = apps['Category'].value_counts()


data = [go.Bar(
        x = num_apps_in_category.index, # index = category name
        y = num_apps_in_category.values, # value = count
)]

print('Number of apps in each category:')
plotly.offline.iplot(data)

Number of categories =  33
Number of apps in each category:


# Distribution of app ratings

In [56]:
# Average rating of apps
avg_app_rating = apps['Rating'].mean()
print('Average app rating = ', avg_app_rating)

# Distribution of apps according to their ratings
data = [go.Histogram(
        x = apps['Rating']
)]

# Vertical dashed line to indicate the average app rating
layout = {'shapes': [{
              'type' :'line',
              'x0': avg_app_rating,
              'y0': 0,
              'x1': avg_app_rating,
              'y1': 1000,
              'line': { 'dash': 'dashdot'}
          }],
          'title': "Distribution of app ratings",
          'xaxis': {'title': "Rating"},
          'yaxis': {'title': "Number of apps"}
          }

plotly.offline.iplot({'data': data, 'layout': layout})

Average app rating =  4.173243045387994
